# APP Options Catalyst Research

## Objective
Identify catalysts that preceded 750%+ intraday moves on APP (AppLovin) options, focusing on:
1. **Ad Industry News** - Sector-wide advertising news affecting APP
2. **Direct Company News** - S&P inclusion, partnerships, earnings
3. **Friday 0DTE Dynamics** - Gamma exposure and pre-market momentum

## Analysis Period
- Recent Fridays: 1/2/26, 12/27/25, 12/20/25, 12/13/25, 12/6/25
- User-identified example: 630 Puts expiring 1/2/26

In [ ]:
# Install dependencies if needed
# !pip install yfinance pandas numpy matplotlib seaborn finnhub-python requests

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8-darkgrid')

print("Libraries loaded successfully!")

## 1. Fetch Historical APP Stock Data

In [ ]:
# Fetch APP stock data (1 year of daily data)
app = yf.Ticker("APP")
app_history = app.history(period="1y")

print(f"APP Stock Data: {len(app_history)} trading days")
print(f"Date Range: {app_history.index[0].date()} to {app_history.index[-1].date()}")
print(f"\nPrice Range: ${app_history['Low'].min():.2f} - ${app_history['High'].max():.2f}")
print(f"Current Price: ${app_history['Close'].iloc[-1]:.2f}")

app_history.tail(10)

In [ ]:
# Calculate daily returns and identify big move days
app_history['Daily_Return'] = app_history['Close'].pct_change() * 100
app_history['Intraday_Range'] = ((app_history['High'] - app_history['Low']) / app_history['Open']) * 100
app_history['Day_of_Week'] = app_history.index.day_name()

# Filter for significant moves (>5% daily move)
big_moves = app_history[abs(app_history['Daily_Return']) > 5].copy()
print(f"\nDays with >5% moves: {len(big_moves)}")
print("\nBig Move Days:")
big_moves[['Open', 'High', 'Low', 'Close', 'Volume', 'Daily_Return', 'Day_of_Week']].sort_values('Daily_Return', ascending=False)

## 2. Analyze Friday Price Action

In [ ]:
# Filter for Fridays only
fridays = app_history[app_history['Day_of_Week'] == 'Friday'].copy()
print(f"Total Fridays in dataset: {len(fridays)}")

# Calculate Friday statistics
print(f"\nFriday Statistics:")
print(f"  Average Daily Return: {fridays['Daily_Return'].mean():.2f}%")
print(f"  Std Dev of Returns: {fridays['Daily_Return'].std():.2f}%")
print(f"  Average Intraday Range: {fridays['Intraday_Range'].mean():.2f}%")
print(f"  Max Intraday Range: {fridays['Intraday_Range'].max():.2f}%")

# Show recent Fridays
print("\n=== Recent Fridays (Last 8 weeks) ===")
fridays.tail(8)[['Open', 'High', 'Low', 'Close', 'Volume', 'Daily_Return', 'Intraday_Range']]

In [ ]:
# Visualize Friday vs Other Days
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Daily returns by day of week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
daily_stats = app_history.groupby('Day_of_Week')['Daily_Return'].agg(['mean', 'std']).reindex(day_order)

axes[0].bar(daily_stats.index, daily_stats['mean'], yerr=daily_stats['std'], capsize=5, color=['gray']*4 + ['red'])
axes[0].set_title('Average Daily Return by Day of Week')
axes[0].set_ylabel('Return (%)')
axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# Intraday range by day of week
range_stats = app_history.groupby('Day_of_Week')['Intraday_Range'].mean().reindex(day_order)
axes[1].bar(range_stats.index, range_stats.values, color=['gray']*4 + ['red'])
axes[1].set_title('Average Intraday Range by Day of Week')
axes[1].set_ylabel('Range (%)')

plt.tight_layout()
plt.show()

## 3. Fetch Intraday Data for Recent Fridays

Analyzing specific Fridays mentioned by user:
- 1/2/26 (Thursday - markets closed Friday 1/3 for National Day of Mourning)
- 12/27/25
- 12/20/25
- 12/13/25
- 12/6/25

In [ ]:
# Fetch intraday data for analysis (5-minute intervals)
# Note: yfinance provides limited intraday history

def fetch_intraday(ticker, days=30):
    """Fetch intraday 5-minute data for the last N days."""
    stock = yf.Ticker(ticker)
    # 5m data available for last 60 days
    data = stock.history(period=f"{days}d", interval="5m")
    return data

# Fetch APP intraday
app_intraday = fetch_intraday("APP", days=30)
print(f"Intraday data points: {len(app_intraday)}")
print(f"Date range: {app_intraday.index[0]} to {app_intraday.index[-1]}")

# Add date column for grouping
app_intraday['Date'] = app_intraday.index.date
app_intraday['Time'] = app_intraday.index.time
app_intraday['Day_of_Week'] = app_intraday.index.day_name()

In [ ]:
# Analyze intraday moves on Fridays
friday_intraday = app_intraday[app_intraday['Day_of_Week'] == 'Friday']

# Calculate intraday statistics per Friday
friday_stats = []
for date in friday_intraday['Date'].unique():
    day_data = friday_intraday[friday_intraday['Date'] == date]
    
    open_price = day_data['Open'].iloc[0]
    close_price = day_data['Close'].iloc[-1]
    high_price = day_data['High'].max()
    low_price = day_data['Low'].min()
    
    friday_stats.append({
        'Date': date,
        'Open': open_price,
        'High': high_price,
        'Low': low_price,
        'Close': close_price,
        'Daily_Return_%': ((close_price - open_price) / open_price) * 100,
        'Intraday_Range_%': ((high_price - low_price) / open_price) * 100,
        'Max_Drawdown_%': ((low_price - open_price) / open_price) * 100,
        'Max_Rally_%': ((high_price - open_price) / open_price) * 100,
    })

friday_df = pd.DataFrame(friday_stats)
print("=== Friday Intraday Analysis ===")
friday_df

## 4. Options Chain Analysis

Analyze current options chain to understand strike distribution and potential 750% movers.

In [ ]:
# Fetch current options chain
app = yf.Ticker("APP")

# Get available expiration dates
expirations = app.options
print("Available Expiration Dates:")
for i, exp in enumerate(expirations[:10]):  # Show first 10
    print(f"  {i+1}. {exp}")

print(f"\n... and {len(expirations) - 10} more expirations")

In [ ]:
# Analyze nearest expiration (likely 0DTE or 1DTE)
if expirations:
    nearest_exp = expirations[0]
    print(f"Analyzing options expiring: {nearest_exp}")
    
    opt_chain = app.option_chain(nearest_exp)
    calls = opt_chain.calls
    puts = opt_chain.puts
    
    current_price = app.history(period='1d')['Close'].iloc[-1]
    print(f"Current APP Price: ${current_price:.2f}")
    
    # Show OTM options
    print(f"\n=== OTM Calls (Strike > ${current_price:.2f}) ===")
    otm_calls = calls[calls['strike'] > current_price].head(10)
    print(otm_calls[['strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'impliedVolatility']])
    
    print(f"\n=== OTM Puts (Strike < ${current_price:.2f}) ===")
    otm_puts = puts[puts['strike'] < current_price].tail(10)
    print(otm_puts[['strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'impliedVolatility']])

In [ ]:
# Calculate potential 750% move scenarios
def calculate_option_gain(entry_price, stock_move_pct, strike, current_stock_price, is_call=True):
    """Simplified option gain calculation (approximation)."""
    new_stock_price = current_stock_price * (1 + stock_move_pct/100)
    
    if is_call:
        intrinsic_new = max(0, new_stock_price - strike)
    else:
        intrinsic_new = max(0, strike - new_stock_price)
    
    if entry_price > 0:
        gain_pct = ((intrinsic_new - entry_price) / entry_price) * 100
    else:
        gain_pct = 0
    
    return gain_pct

print("=== 750% Gain Scenarios ===")
print("\nFor a $0.50 OTM call to gain 750%, it needs to be worth $4.25 at expiration")
print("This means the stock must move beyond the strike by at least $4.25")

# Example calculation
if 'current_price' in dir():
    print(f"\nWith APP at ${current_price:.2f}:")
    strike_5pct_otm = round(current_price * 1.05, 0)
    print(f"  5% OTM Call at ${strike_5pct_otm}")
    print(f"  For 750% gain: Stock needs to reach ${strike_5pct_otm + 4.25:.2f} (if call bought at $0.50)")
    print(f"  That's a {((strike_5pct_otm + 4.25 - current_price) / current_price * 100):.1f}% stock move required")

## 5. News Correlation Analysis

Analyze what news events preceded big move days.

In [ ]:
# Fetch APP news from yfinance
app = yf.Ticker("APP")
news = app.news

print(f"=== Recent APP News ({len(news)} articles) ===")
for article in news[:10]:
    pub_time = datetime.fromtimestamp(article.get('providerPublishTime', 0))
    print(f"\n[{pub_time.strftime('%Y-%m-%d %H:%M')}]")
    print(f"  {article.get('title', 'No title')}")
    print(f"  Source: {article.get('publisher', 'Unknown')}")

In [ ]:
# Fetch ad sector news (META, GOOGL for comparison)
ad_tickers = ['META', 'GOOGL', 'TTD']

print("=== Ad Sector News ===")
for ticker in ad_tickers:
    stock = yf.Ticker(ticker)
    stock_news = stock.news[:3]  # Last 3 articles per stock
    
    print(f"\n--- {ticker} ---")
    for article in stock_news:
        pub_time = datetime.fromtimestamp(article.get('providerPublishTime', 0))
        print(f"  [{pub_time.strftime('%m/%d')}] {article.get('title', 'No title')[:80]}...")

## 6. Specific Analysis: 1/2/26 630 Puts

User-identified example of a large mover.

In [ ]:
# Analyze January 2, 2026 specifically
# Note: This is historical analysis - we need to check if this date has passed

target_date = datetime(2026, 1, 2)
today = datetime.now()

print(f"Target Analysis Date: {target_date.strftime('%Y-%m-%d')}")
print(f"Today's Date: {today.strftime('%Y-%m-%d')}")

if target_date.date() <= today.date():
    print("\n=== Analyzing 1/2/26 Price Action ===")
    
    # Try to get intraday data for that specific date
    # This may require the date to be within the 60-day yfinance limit
    try:
        jan2_data = app_intraday[app_intraday['Date'] == target_date.date()]
        if len(jan2_data) > 0:
            print(f"\nIntraday data points for 1/2/26: {len(jan2_data)}")
            print(f"Open: ${jan2_data['Open'].iloc[0]:.2f}")
            print(f"High: ${jan2_data['High'].max():.2f}")
            print(f"Low: ${jan2_data['Low'].min():.2f}")
            print(f"Close: ${jan2_data['Close'].iloc[-1]:.2f}")
            
            # Calculate what happened to 630 puts
            open_price = jan2_data['Open'].iloc[0]
            low_price = jan2_data['Low'].min()
            
            print(f"\n=== 630 Put Analysis ===")
            print(f"Strike: $630")
            print(f"Stock opened at: ${open_price:.2f}")
            print(f"Stock low was: ${low_price:.2f}")
            
            if low_price < 630:
                intrinsic_at_low = 630 - low_price
                print(f"Put went ITM by: ${intrinsic_at_low:.2f}")
            else:
                print(f"Put remained OTM (stock didn't drop below $630)")
        else:
            print("No intraday data available for 1/2/26 in current dataset")
    except Exception as e:
        print(f"Error fetching data: {e}")
else:
    print("\n1/2/26 is in the future - cannot analyze yet")

In [ ]:
# Plot intraday price action for 1/2/26 if available
if 'jan2_data' in dir() and len(jan2_data) > 0:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    ax.plot(jan2_data.index, jan2_data['Close'], label='Price', linewidth=2)
    ax.axhline(y=630, color='red', linestyle='--', label='$630 Strike', linewidth=2)
    
    ax.fill_between(jan2_data.index, jan2_data['Low'], jan2_data['High'], alpha=0.3)
    
    ax.set_title('APP Intraday Price Action - January 2, 2026', fontsize=14)
    ax.set_xlabel('Time')
    ax.set_ylabel('Price ($)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 7. Summary & Signal Definition

Based on the analysis above, define the catalyst signals.

In [ ]:
# Summary Statistics
print("=" * 60)
print("CATALYST RESEARCH SUMMARY")
print("=" * 60)

print("\n1. FRIDAY DYNAMICS")
print("-" * 40)
if 'friday_df' in dir() and len(friday_df) > 0:
    print(f"   Average Friday Return: {friday_df['Daily_Return_%'].mean():.2f}%")
    print(f"   Average Intraday Range: {friday_df['Intraday_Range_%'].mean():.2f}%")
    print(f"   Max Rally (any Friday): {friday_df['Max_Rally_%'].max():.2f}%")
    print(f"   Max Drawdown (any Friday): {friday_df['Max_Drawdown_%'].min():.2f}%")

print("\n2. BIG MOVE ANALYSIS")
print("-" * 40)
if 'big_moves' in dir() and len(big_moves) > 0:
    friday_big_moves = big_moves[big_moves['Day_of_Week'] == 'Friday']
    print(f"   Total days with >5% moves: {len(big_moves)}")
    print(f"   Friday big moves: {len(friday_big_moves)}")
    print(f"   % of big moves on Friday: {len(friday_big_moves)/len(big_moves)*100:.1f}%")

print("\n3. PROPOSED SIGNAL THRESHOLDS")
print("-" * 40)
print("   [To be refined based on complete data analysis]")
print("   - Ad Industry News: Major headlines from META/GOOGL")
print("   - Direct APP News: Any company-specific announcement")
print("   - 0DTE Setup: Friday with intraday range > X%")

In [ ]:
# Export findings for use in signal implementation
findings = {
    'ticker': 'APP',
    'analysis_date': datetime.now().strftime('%Y-%m-%d'),
    'catalysts': [
        {
            'name': 'Ad Industry News',
            'description': 'Major news from META, GOOGL, or digital advertising sector',
            'keywords': ['digital advertising', 'ad spend', 'programmatic', 'ad revenue'],
            'related_tickers': ['META', 'GOOGL', 'TTD', 'MGNI', 'PUBM']
        },
        {
            'name': 'Direct Company News',
            'description': 'APP-specific announcements',
            'triggers': ['S&P inclusion', 'earnings', 'partnerships', 'acquisitions']
        },
        {
            'name': 'Friday 0DTE Setup',
            'description': 'High gamma exposure on Friday expirations',
            'conditions': ['Friday market day', '0DTE expiration available', 'pre-market momentum']
        }
    ],
    'entry_criteria': {
        'days': ['Thursday', 'Friday'],
        'dte': [0, 1, 2],
        'strike_type': ['OTM', 'Deep OTM'],
        'target_gain': 750  # percent
    }
}

print("\nFindings exported to 'findings' dictionary")
print("Ready for signal implementation phase")

## Next Steps

1. **Refine thresholds** - Run this notebook with live data to establish statistical thresholds
2. **Backtest signals** - Test catalyst signals against historical 750%+ moves
3. **Implement in production** - Port findings to `src/signals/` modules
4. **Set up news monitoring** - Configure Finnhub/NewsAPI for real-time catalyst detection